In [4]:
import sys

"""

The Elves each reach into their pack and pull out a tiny plant. The plants rely on important nutrients from the ash, so they can't be planted too close together.

There isn't enough time to let the Elves figure out where to plant the seedlings themselves; you quickly scan the grove (your puzzle input) and note their positions.

For example:

....#..
..###.#
#...#.#
.#...##
#.###..
##.#.##
.#..#..

The scan shows Elves # and empty ground .; outside your scan, more empty ground extends a long way in every direction. The scan is oriented so that north is up; orthogonal directions are written N (north), S (south), W (west), and E (east), while diagonal directions are written NE, NW, SE, SW.

The process consists of some number of rounds during which Elves alternate between considering where to move and actually moving.

During the first half of each round, each Elf considers the eight positions adjacent to themself. If no other Elves are in one of those eight positions, the Elf does not do anything during this round. Otherwise, the Elf looks in each of four directions in the following order and proposes moving one step in the first valid direction:

If there is no Elf in the N, NE, or NW adjacent positions, the Elf proposes moving north one step.
If there is no Elf in the S, SE, or SW adjacent positions, the Elf proposes moving south one step.
If there is no Elf in the W, NW, or SW adjacent positions, the Elf proposes moving west one step.
If there is no Elf in the E, NE, or SE adjacent positions, the Elf proposes moving east one step.

After each Elf has had a chance to propose a move, the second half of the round can begin. Simultaneously, each Elf moves to their proposed destination tile if they were the only Elf to propose moving to that position. If two or more Elves propose moving to the same position, none of those Elves move.

Finally, at the end of the round, the first direction the Elves considered is moved to the end of the list of directions. For example, during the second round, the Elves would try proposing a move to the south first, then west, then east, then north. On the third round, the Elves would first consider west, then east, then north, then south.

---
Round sequence:
1. All Elves consider their 8 adjacent positions.
1.a. If they are solo, they end their round
1.b. If they are not solo, they propose a move to the first valid direction, in the following order:
     N if no Elf in N, NE, NW
     S if no Elf in S, SE, SW
     W if no Elf in W, NW, SW
     E if no Elf in E, NE, SE

2. All Elves move to their proposed destination tile if they were the only Elf to propose moving to that position.

3. The order of the decision is rotated by one, so that the first direction the Elves considered is moved to the end of the list of directions.
---

Rounds continue until no Elves are able to move (they have no elf in any of the 8 adjacent positions).
"""

def parse_file_to_grid(filename):
    with open(filename, 'r') as f:
        grid = [list(line.strip()) for line in f.readlines()]
    return grid

def print_grid(grid):
    for row in grid:
        print(''.join(row))

In [51]:
def get_elf_positions_from_grid(grid):
    elf_positions = []
    for row_idx, row in enumerate(grid):
        for col_idx, col in enumerate(row):
            if col == '#':
                elf_positions.append((row_idx, col_idx))
    return elf_positions

def grid_for_elf_positions(elf_positions):
    max_row = max(elf_positions, key=lambda x: x[0])[0]
    max_col = max(elf_positions, key=lambda x: x[1])[1]
    grid = [['.' for _ in range(max_col + 1)] for _ in range(max_row + 1)]
    for row, col in elf_positions:
        grid[row][col] = '#'
    return grid

def get_bounding_area(elf_positions):
    min_row = min(elf_positions, key=lambda x: x[0])[0]
    max_row = max(elf_positions, key=lambda x: x[0])[0]
    min_col = min(elf_positions, key=lambda x: x[1])[1]
    max_col = max(elf_positions, key=lambda x: x[1])[1]
    return (max_col - min_col + 1) * (max_row - min_row + 1)

In [52]:
import enum

class Direction(enum.Enum):
    NW = (-1, -1)
    N = (-1, 0)
    NE = (-1, 1)
    W = (0, -1)
    E = (0, 1)
    SW = (1, -1)
    S = (1, 0)
    SE = (1, 1)

    def offset_from(self, position):
        row, col = position
        return (row + self.value[0], col + self.value[1])

class DecisionOrder:
    NORTHWARD = [Direction.NW, Direction.N, Direction.NE]
    SOUTHWARD = [Direction.SW, Direction.S, Direction.SE]
    WESTWARD = [Direction.NW, Direction.W, Direction.SW]
    EASTWARD = [Direction.NE, Direction.E, Direction.SE]

    ALL_DIRECTIONS = NORTHWARD + SOUTHWARD + WESTWARD + EASTWARD

    INITIAL_ORDER = NORTHWARD, SOUTHWARD, WESTWARD, EASTWARD

    def __init__(self):
        self.directions = DecisionOrder.INITIAL_ORDER

    def rotate(self):
        self.directions = self.directions[1:] + self.directions[:1]

    def __str__(self):
        return str(self.directions)

def get_adjacent_positions(position, directions):
    return [d.offset_from(position) for d in directions]

def elves_in_positions(positions, elf_positions):
    return [p for p in positions if p in elf_positions]

In [53]:
# Return a dict keyed by current elf position with a value of proposed position
def get_proposed_positions(elf_positions, decision_order):
    proposed_positions = {}
    for elf_position in elf_positions:
        adjacent_positions = get_adjacent_positions(elf_position, decision_order.ALL_DIRECTIONS)
        if not elves_in_positions(adjacent_positions, elf_positions):
            proposed_positions[elf_position] = elf_position
            continue
        for directions in decision_order.directions:
            adjacent_positions = get_adjacent_positions(elf_position, directions)
            if not elves_in_positions(adjacent_positions, elf_positions):
                proposed_positions[elf_position] = directions[1].offset_from(elf_position)
                break
        if elf_position not in proposed_positions:
            proposed_positions[elf_position] = elf_position
    return proposed_positions

# Return a tuple of an array with elf positions updated for non-conflicting moves and the number of moves made
def move_to_proposals(elf_positions, proposals):
    valid_proposals = {k: v for k, v in proposals.items() if list(proposals.values()).count(v) == 1 and k != v}
    return ([valid_proposals.get(p, p) for p in elf_positions], len(valid_proposals))

In [58]:
decisions = DecisionOrder()
elf_positions = get_elf_positions_from_grid(parse_file_to_grid('./day23-input.txt'))
moves = len(elf_positions)
round = 0

# print("Initial position")
# print_grid(grid_for_elf_positions(elf_positions))
# print()

while moves != 0:
    elf_positions, moves = move_to_proposals(elf_positions, get_proposed_positions(elf_positions, decisions))
    decisions.rotate()
    round += 1

    print("Round {} - {} moves".format(round, moves))
    # print_grid(grid_for_elf_positions(elf_positions))
    # print()

empty_tiles = get_bounding_area(elf_positions) - len(elf_positions)

print("Empty tiles: {}".format(empty_tiles))

# Optimization this time was to just let the computer do its thing for 35 minutes...

Round 1 - 675 moves
Round 2 - 469 moves
Round 3 - 396 moves
Round 4 - 399 moves
Round 5 - 388 moves
Round 6 - 402 moves
Round 7 - 391 moves
Round 8 - 432 moves
Round 9 - 429 moves
Round 10 - 433 moves
Round 11 - 409 moves
Round 12 - 444 moves
Round 13 - 425 moves
Round 14 - 467 moves
Round 15 - 456 moves
Round 16 - 487 moves
Round 17 - 480 moves
Round 18 - 505 moves
Round 19 - 488 moves
Round 20 - 511 moves
Round 21 - 484 moves
Round 22 - 535 moves
Round 23 - 510 moves
Round 24 - 535 moves
Round 25 - 507 moves
Round 26 - 529 moves
Round 27 - 520 moves
Round 28 - 583 moves
Round 29 - 554 moves
Round 30 - 586 moves
Round 31 - 565 moves
Round 32 - 619 moves
Round 33 - 569 moves
Round 34 - 578 moves
Round 35 - 530 moves
Round 36 - 600 moves
Round 37 - 565 moves
Round 38 - 596 moves
Round 39 - 580 moves
Round 40 - 640 moves
Round 41 - 612 moves
Round 42 - 646 moves
Round 43 - 615 moves
Round 44 - 648 moves
Round 45 - 608 moves
Round 46 - 639 moves
Round 47 - 632 moves
Round 48 - 658 moves
R